In [1]:
photo_paths = '/home/kyle/datasets/sketchy-photos/images/'

In [2]:
import caffe
import numpy as np

In [3]:
model_root = '../models/triplet_googlenet/Triplet_googlenet_'
pretrained_model = model_root + 'final.caffemodel' 
image_proto = model_root + 'imagedeploy.prototxt'

In [4]:
caffe.set_mode_gpu()
img_net = caffe.Net(image_proto, pretrained_model, caffe.TEST)

In [5]:
output_layer_image = 'pool5/7x7_s1_p'

In [6]:
transformer = caffe.io.Transformer({'data': np.shape(img_net.blobs['data'].data)})
transformer.set_mean('data', np.array([104, 117, 123]))
transformer.set_transpose('data',(2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

In [7]:
import os
import fnmatch

# extensions can be a single tring like '.png' or '.jpg'
# or a list of extensions. they should all be lowercase
# but the . is important.
def list_all_files(directory, extensions=None):
    for root, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            base, ext = os.path.splitext(filename)
            joined = os.path.join(root, filename)
            if extensions is None or ( len(ext) and ext.lower() in extensions ):
                yield joined
                
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [15]:
from os.path import basename
paths = list(list_all_files(photo_paths, '.jpg'))
np.savetxt('filenames.txt', [basename(path) for path in paths], fmt='%s')
len(paths)

12500

In [9]:
from tqdm import tqdm

def get_feats(paths, batch_size=1):
    feats = []
    for chunk in tqdm(chunks(paths, batch_size)):
        imgs = [transformer.preprocess('data', caffe.io.load_image(path)) for path in chunk]
        imgs = np.asarray(imgs)
        if img_net.blobs['data'].data.shape != imgs.shape:
            img_net.blobs['data'].reshape(*imgs.shape)
        results = img_net.forward(data=imgs)[output_layer_image]
        results = results.reshape(results.shape[:2])
        feats.extend(np.copy(results))
    return np.asarray(feats)

In [10]:
feats = get_feats(paths, 128)
np.save('features.npy', feats)
feats.shape

98it [05:39,  3.22s/it]


(12500, 1024)